In [34]:
import pandas as pd


In [35]:
csv_path = "/zi/flstorage/HITKIP/datasets/NAKO_01_2024/NAKO/sourcedata/NAKO-711_Studiendaten/NAKO-711_export_Auswertung_export_baseline.csv"
merged_df = pd.read_csv(csv_path, sep=";")
merged_df["ID"] = merged_df["ID"].astype(str)

In [36]:
for col in merged_df.columns:
    print(col)

ID
basis_sex
basis_age
basis_ageclass
a_ethn1
a_ethn2
a_ses_mig_status
a_ses_aussiedler
a_ses_turkey
a_betablock
idom_count_med
unt192_status
a_apo_count
a_apo_yearsfirst
a_apo_yearslast
a_epi_yearsfirst
a_park_yearsfirst
a_diag_epilep
a_diag_parkinson
d_an_mem1
d_an_mem2
d_an_mem4
d_an_mem5
d_an_minia1
d_an_minib1
d_an_minib2
d_an_minif1
d_an_minif2
d_an_minif3_a
d_an_minif4
d_an_neudep2
d_an_neuangst2
d_an_mem3
d_an_mem3_j
d_an_mem3_m
d_an_minid1a
d_an_minid1b
d_an_minid2
d_an_minid3a
d_an_minid3b
d_an_minid4
d_an_minid5
d_an_minid6
d_an_minid7
d_an_neu_1
d_an_neu_2
d_an_neu_3
d_an_neu_4
d_an_neu_5
d_an_neu_6
a_ses_famst
a_ses_partner
a_ses_househ
a_ses_child
a_ses_ewstat
a_ses_beruf
a_ses_workh
a_ses_el_datum
a_ses_el_seit_j
a_ses_el_gesamt
a_ses_rentenalter
a_kldb_code
a_kldb_major
a_kldb_anf
a_kldb_fuehr
a_kldb_seg
a_kldb_sek
a_isco_code
a_isco_skill
a_isei
a_siops
a_selfemp
a_nempl
a_esec
a_ses_isced97_years
a_ses_isced97_level
a_ses_isced97_cat
a_ses_isced97_cat_ye
a_ses_deutsch

In [37]:

# Filter the dataframe
merged_df = merged_df[merged_df["a_diag_epilep"] == 0]
merged_df = merged_df[merged_df["a_diag_parkinson"] == 0]
filtered_df = merged_df[
    (merged_df["a_emo_phq9_kat"] != 7777) &
    (merged_df["a_emo_gad7_dia"] != 9999) &
    (merged_df["a_alk_rein"] > 0) &
    (merged_df["a_alk_konsum"] != -99) &
    (merged_df["a_alk_audit_c"] != -99) &
    (merged_df["a_smok_stat_noint"].isin([1, 2])) &
    (merged_df["a_anthro_bmi"] != 7777)
    
]

all_diagnoses = [
    ("depression_and_angst", 2), ("depression_and_angst", 3), ("depression_and_angst", 4),
    ("depression", 2), ("depression", 3), ("depression", 4),
    ("angst", 2), ("angst", 3), ("angst", 4),
    ("alkohol", 70), ("alkohol", 80), ("alkohol", 90),
    ("alkohol_audit", 9), ("alkohol_audit", 10)  # ,  ("alkohol_audit", 11)
]

for diagnosis_crit, thresh in all_diagnoses:
    if diagnosis_crit == "depression_and_angst":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if (row['a_emo_phq9_kat'] >= thresh) or \
                             (row['a_emo_gad7_dia'] >= thresh) else 0,
            axis=1
        )
    elif diagnosis_crit == "depression":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_emo_phq9_kat'] >= thresh else 0,
            axis=1
        )
    elif diagnosis_crit == "angst":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_emo_gad7_dia'] >= thresh else 0,
            axis=1
        )
    elif diagnosis_crit == "alkohol":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_alk_rein'] >= thresh and row['a_alk_konsum'] == 2 else 0,
            axis=1
        )
    elif diagnosis_crit == "alkohol_audit":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_alk_audit_c'] >= thresh else 0,
            axis=1
        )
        

/tmp/ipykernel_2110521/1522020995.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
/tmp/ipykernel_2110521/1522020995.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
/tmp/ipykernel_2110521/1522020995.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [38]:
#wen all diagnose are zero set health col to 1
filtered_df["healthy"] = filtered_df.apply(
    lambda row: 1 if sum([row[f'diagnosis_{diagnosis_crit}_{thresh}'] for diagnosis_crit, thresh in all_diagnoses]) == 0 else 0,
    axis=1
)

/tmp/ipykernel_2110521/1333744814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["healthy"] = filtered_df.apply(


In [39]:
Healthy_people = filtered_df[filtered_df["healthy"] == 1]

In [40]:
age_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages_all_ids.csv")
if "Unnamed: 0" in age_df.columns:
    age_df.drop(columns=["Unnamed: 0"], inplace=True)
len(age_df)

30927

In [41]:
#merge the ID for healthy people
age_df["ID"] = age_df["ID"].astype(str)
merged_df = pd.merge(age_df, Healthy_people, on="ID", how="inner")
final_all_ages = merged_df[age_df.columns]

In [42]:
len(final_all_ages)

16364

In [43]:
final_all_ages.rename(columns={"Age": "label_Age"}, inplace=True)

/tmp/ipykernel_2110521/1441495901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_all_ages.rename(columns={"Age": "label_Age"}, inplace=True)


In [44]:
#save the merged df as all ages.csv
final_all_ages.to_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages_healthy.csv", index=False)

In [45]:
demographics = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/NAKO_demographics_covariates.csv")
len(demographics)

30927

In [46]:
demographics

,ID,Sex,Age,Site
0,100000,1,28,31
1,100001,0,48,31
2,100002,0,44,11
3,100003,0,50,21
4,100004,0,65,11
...,...,...,...,...
30922,130924,0,54,11
30923,130925,0,48,11
30924,130926,1,58,11
30925,130927,1,63,81


In [47]:
#merges with Healtey people
demographics["ID"] = demographics["ID"].astype(str)
merged_df = pd.merge(demographics, Healthy_people, on="ID", how="inner")
final_demographics = merged_df[demographics.columns]

In [48]:
#Dorp outlier for BMI a_anthro_bmi
z_scores = (merged_df['a_anthro_bmi'] - merged_df['a_anthro_bmi'].mean()) / merged_df['a_anthro_bmi'].std()
threshold = 3
df_bmi = merged_df[abs(z_scores) <= threshold]
df_bmi = df_bmi[["ID","a_anthro_bmi"]]

final_demographics = pd.merge(final_demographics, df_bmi, on="ID", how="inner")

In [49]:
final_demographics.rename(columns={"a_anthro_bmi": "BMI"}, inplace=True)

In [50]:
len(final_demographics)
final_demographics.to_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/NAKO_demographics_covariates_subset_healthy.csv", index=False)